# Optimal Control

In [1]:
import os
import numpy as np
from numpy.testing import assert_array_equal, assert_array_almost_equal
import matplotlib.pyplot as plt
from casadi import (
    MX, DM, Function, Opti, sparsify,
    vec, vcat, vertcat, horzcat, blockcat, sum1, sum2, sumsqr,
    gradient, jacobian, hessian
)

In [2]:
data_dir = 'data'

## 1. Solve Multiple Shooting Problem

In [3]:
# State transition function
uk = MX.sym('uk')
xk = MX.sym('xk')
F = Function('F', [xk, uk], [xk ** 2 + uk], ['xk', 'uk'], ['xkp1'])
F

Function(F:(xk,uk)->(xkp1) MXFunction)

Discrete optimal control problem:

$$
\DeclareMathOperator*{\minimize}{minimize}
\begin{aligned}
\minimize_{x_1,x_2,...,x_{N+1},u_1,u_2,...,u_N} \quad & \sum_{k=1}^{N}{u_k^2} + \sum_{k=1}^{N+1}{x_k^2} \\
\text{subject to} \quad & F(x_k,u_k) = x_{k+1}, \quad k=1, ..., N \\
                  & x_1 = 2 \\
                  & x_{N+1} = 3 \\
                  & x_k \ge 0, \quad k=1, ..., N+1 \\
\end{aligned}
$$


In [4]:
opti_MS = Opti()
N = 4
X = opti_MS.variable(N + 1)
U = opti_MS.variable(N)

# Define objective function
f = sum1(X ** 2) + sum1(U ** 2)
# or sumsqr(X) + sumsqr(U)

# Define constraints
for k in range(1, N + 1):
    opti_MS.subject_to(X[k] == F(X[k-1], U[k-1]))
opti_MS.subject_to(X[0] == 2)  # initial constraint
opti_MS.subject_to(X[-1] == 3)  # final constraint
opti_MS.subject_to(X >= 0)  # path constraint

opti_MS.minimize(f)

In [5]:
opti_MS.solver('ipopt')
sol_MS = opti_MS.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       14
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:        9
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality c

In [6]:
sol_MS.value(U)

array([-2.70380638, -0.54297936,  0.26125189,  0.58403971])

In [7]:
assert_array_almost_equal(sol_MS.value(U), [-2.7038, -0.5430, 0.2613, 0.5840], decimal=4)

## 2. Adapt to single-shooting problem

In [15]:
opti_SS = Opti()

# Objective function
N = 4
x0 = 2  # initial state
U = opti_SS.variable(N)
X = [x0]
for k in range(1, N + 1):
    X.append(F(X[k-1], U[k-1]))
X = vertcat(*X)
f = sum1(X ** 2) + sum1(U ** 2)

# Define constraints
opti_SS.subject_to(X[-1] == 3)
opti_SS.subject_to(X[1:-1] >= 0)

opti_SS.minimize(f)

In [16]:
X[-1]

MX(F(F(F(F(2, opti2_x_1[0]){0}, opti2_x_1[1]){0}, opti2_x_1[2]){0}, opti2_x_1[3]){0})

In [17]:
opti_SS.solver('ipopt')
sol_SS = opti_SS.solve()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.2950331e+09 6.55e+04 1.21e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [18]:
sol_SS.value(U)

array([-2.70380008, -0.54298561,  0.26123341,  0.58403084])

In [19]:
assert_array_almost_equal(sol_SS.value(U), [-2.7038, -0.5430, 0.2613, 0.5840], decimal=4)

## 3. Plot sparsities of the Hessian

In [12]:
opti_MS.f.shape, opti_MS.x.shape

((1, 1), (9, 1))

In [13]:
H_MS = hessian(opti_MS.f, opti_MS.x)[0]
H_MS.shape

(9, 9)

In [14]:
H_MS.sparsity().spy()

*........
.*.......
..*......
...*.....
....*....
.....*...
......*..
.......*.
........*


In [15]:
opti_SS.f.shape, opti_SS.x.shape

((1, 1), (4, 1))

In [16]:
H_SS = hessian(opti_SS.f, opti_SS.x)[0]
H_SS.shape

(4, 4)

In [17]:
H_SS.sparsity().spy()

****
****
****
****


In [18]:
constraint_J_MS = jacobian(opti_MS.g, opti_MS.x)
constraint_J_MS.shape

(11, 9)

In [19]:
constraint_J_MS.sparsity().spy()

**...*...
.**...*..
..**...*.
...**...*
*........
....*....
*........
.*.......
..*......
...*.....
....*....


In [20]:
constraint_J_SS = jacobian(opti_SS.g, opti_SS.x)
constraint_J_SS.shape

(6, 4)

In [21]:
constraint_J_SS.sparsity().spy()

****
....
*...
**..
***.
****


In multiple-shooting there are more decision variables but each decision variable is only dependent on the states and inputs in the previous time-step, therefore the Hessian is sparse.  In this example, the dependency on the previous input is linear therefore the Hessian has no cross-terms.

In single-shooting, the future states are dependent on the inputs in all previous time steps so the Hessian has many cross-terms (i.e. dense). The way the objective is defined, the relationship between f and u is highly non-linear and therefore all the terms in the Hessian are non-zero.

## 4. Banded constraint Jacobian

In [22]:
# To make the constraint Jacobian more banded, arrange the xk and uk terms next to each other:

opti = Opti()
x0 = opti.variable()
opti.subject_to(x0==2)

U = []
X = [x0]
for k in range(1, N + 1):
    xkm1 = X[k-1]
    uk = opti.variable()
    xk = opti.variable()
    opti.subject_to(F(xkm1, uk) == xk)
    if k < N:
        opti.subject_to(xk >= 0)
    U.append(uk)
    X.append(xk)
opti.subject_to(X[-1] == 3)

U = vcat(U)
X = vcat(X)

In [23]:
opti.minimize(sumsqr(U) + sumsqr(X))

opti.solver('ipopt')
sol = opti.solve()

Usol = sol.value(U) # should be [-2.7038;-0.5430;0.2613;0.5840]
print(Usol)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       14
Number of nonzeros in inequality constraint Jacobian.:        3
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:        9
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        3
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 3.00e+00 3.33e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [24]:
# In fact the ordering didn't significantly affected the convergence

In [25]:
assert_array_almost_equal(sol.value(U), [-2.7038, -0.5430, 0.2613, 0.5840], decimal=4)

In [26]:
H = hessian(opti.f, opti.x)[0]
H.shape

(9, 9)

In [27]:
H.sparsity().spy()

*........
.*.......
..*......
...*.....
....*....
.....*...
......*..
.......*.
........*


In [28]:
constraint_J = jacobian(opti.g, opti.x)
constraint_J.shape

(9, 9)

In [29]:
constraint_J.sparsity().spy()

*........
***......
..*......
..***....
....*....
....***..
......*..
......***
........*


## 5. Which transcription scales best for large N?

The Hessian becomes part of the KKT matrix, which must be factorized/inverted by the NLP solver.

Inverting a dense system of shape N-by-N costs $O(N^3)$.

Solvers can take advantage of the sparsity of the Hessian in multiple-shooting (MS) and so it scales better to long time horizons (i.e. large $N$).

## 6. Numerical values of the constraint Jacobian at initialization

In [30]:
sol_MS.value(constraint_J_MS, opti_MS.initial()).toarray()

array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]])

In [31]:
sol_SS.value(constraint_J_SS, opti_SS.initial()).toarray()

array([[1.31072e+05, 1.63840e+04, 5.12000e+02, 1.00000e+00],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [8.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00],
       [2.56000e+02, 3.20000e+01, 1.00000e+00, 0.00000e+00],
       [1.31072e+05, 1.63840e+04, 5.12000e+02, 1.00000e+00]])

For single-shooting (SS) the initial values of the states can get high (especially if for example, the system is unstable), therefore initialization can be tricky unless you already have a reasonable intitial solution.

## 8. Compare the number of iterations

In [40]:
sol_SS.stats()['iter_count']

78

In [41]:
sol_MS.stats()['iter_count']

9

Order of polynomial $F(x, 0)$ is 1 (linear)

Order of polynomial $F(F(x, 0), 0)$ is 2 (quadratic)

Therefore as the prediction horizon $N$ increases the order of the resulting polynomial gets higher.

Since each iteration involves a linearization, which is an approximation, the non-linearities will reduce the speed of convergence, requiring more iterations to converge to the solution.

## 9. Use of a reasonable initial guess

Gaving an initial guess which is close (in some sense) to the solutioin helps because the initial value of the objective function will be lower and therefore less itrations will be needed to reach the solution.

## 10. Multiple shooting with a 2x3 system

In [18]:
A = sparsify(DM([[1, 0.1, 0.2], [2, 0.3, 0.4], [6, 1, 3]]))
A

DM(
[[1, 0.1, 0.2], 
 [2, 0.3, 0.4], 
 [6, 1, 3]])

In [19]:
B = sparsify(DM([[1, 0], [0, 1], [2, 1]]))
# or B = sparsify(blockcat([[1,0],[0,1],[2,1]]))
B

DM(
[[1, 00], 
 [00, 1], 
 [2, 1]])

In [20]:
xk = MX.sym('xk', 3)
uk = MX.sym('uk', 2)
F = Function('F', [xk, uk], [A @ xk + B @ uk], ['xk', 'uk'], ['xkp1'])
# or simply, F = lambda x,u: A @ x + B @ u
F

Function(F:(xk[3],uk[2])->(xkp1[3]) MXFunction)

In [21]:
opti_MS = Opti()
N = 4
X = opti_MS.variable(N + 1, 3)
U = opti_MS.variable(N, 2)

# Define objective function
f = sumsqr(X) + sumsqr(U)

# Define constraints
for k in range(1, N + 1):
    xkm1, ukm1 = X[k-1, :].T, U[k-1, :].T
    opti_MS.subject_to(X[k, :] == F(xkm1, ukm1).T)
opti_MS.subject_to(X[0, :] == DM([1, 2, 3]).T)
opti_MS.subject_to(X[-1, :] == DM.zeros(3).T)

opti_MS.minimize(f)

In [22]:
opti_MS.solver('ipopt')
sol = opti_MS.solve()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       70
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       23

Total number of variables............................:       23
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       18
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 3.00e+00 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [23]:
sol.value(U[0, :])

array([-4.28944779, -4.19298955])

In [24]:
assert_array_almost_equal(sol.value(U[0, :]), [-4.2894, -4.1930], decimal=4)